Đọc file test 

In [ ]:
import pandas as pd
# Read the CSV file
data = pd.read_csv('Vongck.csv')

0                     Tiến sĩ câu giờ, hoàng đế gây mê\n
1      Mới học 2 bữa thôi là đã nghiện cách giảng dạy...
2      kêu giơ tay phát biểu cộng điểm nhưng có cái n...
3               thầy mãi là number one trong lòng em !\n
4      Thầy như là người được chọn, sỡ hữu haki bá vư...
                             ...                        
495    THầy là người ru ngủ giỏi nhất em từng gặp <3 ...
496    Cô rất dễ tính, cho bài hơi khó nhưng bù lại g...
497                                  ông vua đọc slide\n
498                     Giọng giảng bài to rõ, dễ hiểu\n
499    Không quá nổi trội so với giáo viên trường mìn...
Name: comment, Length: 500, dtype: object

Preprocesing


In [ ]:
pip install nltk 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 19.0 MB/s eta 0:00:00


Loại bỏ những từ không cần thiết

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import codecs
# download required resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
def create_stopwordlist():
    f = codecs.open('vietnamese_stopword.txt', encoding='utf-8')
    datavn = []
    null_data = []
    for i, line in enumerate(f):
        line = repr(line)
        line = line[1:len(line)-3]
        datavn.append(line)
    return datavn
def remove_numbers(text):
    return re.sub(r'\d+', '', text)
def remove_zw_words(tokens):
    return [token for token in tokens if not re.search(r'[zw]', token)]
column_name = 'comment'
data[column_name] = data[column_name].astype(str)
# lowercase
data[column_name] = data[column_name].apply(lambda x: x.lower())

# punctuation removal
data[column_name] = data[column_name].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# tokenization
data[column_name] = data[column_name].apply(lambda x: word_tokenize(x))

# stopword removal
stop_words = create_stopwordlist()
data[column_name] = data[column_name].apply(lambda x: [token for token in x if not token in stop_words])
data[column_name] = data[column_name].apply(lambda x: [remove_numbers(token) for token in x])
# remove words containing 'z' or 'w'
data[column_name] = data[column_name].apply(remove_zw_words)
# rejoin tokens into text
data[column_name] = data[column_name].apply(lambda x: ' '.join(x))

# save the preprocessed data to a new CSV file
# data.to_csv('preprocessed.csv', index=False)
# data['comment']=data['comment'].apply(normalizer)
# data.to_csv('processed_data.csv', index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Chỉnh sửa chính tả 

In [ ]:
from collections import Counter
from keras.models import load_model
from nltk.tokenize import word_tokenize
from nltk import ngrams,word_tokenize
import numpy as np
import re
import unidecode
import string
from tqdm import tqdm

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

model = load_model("spell.h5")
model.make_predict_function()



NGRAM=5  
MAXLEN=40 
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
accepted_char=list((string.digits + ''.join(letters)))

def extract_phrases(text):
    pattern = r'\w[\w ]*|\s\W+|\W+'
    return re.findall(pattern, text)

def encoder_data(text, maxlen=MAXLEN):
        text = "\x00" + text
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x
      
def decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)


def nltk_ngrams(words, n=5):
    return ngrams(words.split(), n)
      
def guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([encoder_data(text)]), verbose=0)
    return decoder_data(preds[0]).strip('\x00')

def correct(sentence):
    for i in sentence:
        if i not in accepted_char:
            sentence=sentence.replace(i," ")
    ngrams = list(nltk_ngrams(sentence, n=NGRAM))
    guessed_ngrams = list(guess(ngram) for ngram in ngrams)
    candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
    for nid, ngram in (enumerate(guessed_ngrams)):
        for wid, word in (enumerate(re.split(' +', ngram))):
            candidates[nid + wid].update([word])
    output = ' '.join(c.most_common(1)[0][0] for c in candidates)     
    return output



for i in range(len(data['comment'])):
    try:
        corrected = correct(data['comment'][i]) # áp dụng hàm correct vào câu hiện tại
        data['comment'][i] = corrected # cập nhật phần tử hiện tại trong danh sách ban đầu bằng câu đã sửa chữa
    except:
        pass # nếu có lỗi xảy ra, bỏ qua câu hiện tại và tiếp tục vòng lặp


#Build Model and Training#

In [ ]:
import pandas as pd

train_data = pd.read_csv('trainLK.csv')

In [ ]:
# train_data = get_train_data('data_phase_1.txt')
X_train, y_train = train_data['comment'], train_data['label']

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
vectorizer = CountVectorizer(max_features=10000)
classifier = LinearSVC(C=1, class_weight='balanced')
model_min = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("classifier", classifier),
    ]
)

model_min.fit(X_train, y_train)


Pipeline(steps=[('vectorizer', CountVectorizer(max_features=10000)),
                ('classifier', LinearSVC(C=1, class_weight='balanced'))])

#RRESULT#

In [ ]:
X_test, y_test = data['comment'], data['label']
y_predict = model_min.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_pred=y_predict, y_true=y_test, average='weighted')
recall = recall_score(y_pred=y_predict, y_true=y_test, average='weighted')
f1 = f1_score(y_pred=y_predict, y_true=y_test, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
filename = 'submit.txt'
with open(filename, 'w') as f:
    for item in y_predict:
        f.write("%s\n" % item)
